In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from hdfisher import fisher
from hd_pk import cmb_from_pk

# Examples to calculate new Fisher matrices

There are two main steps in the process of calculating a Fisher matrix:
1. Calculating the numerical derivatives of the power spectra with respect to each varied parameter, and
2. Using these derivatives, along with a covariance matrix for the spectra, to calculate the Fisher matrix itself.

Below, we will give a few examples of how to calculate the derivatives in step (1). They will be listed in increasing order of complexity. Then we will show how to use them in step (2) to calculate a Fisher matrix.



---

---

# Step 1: numerical derivatives

The numerical derivatives of the CMB $TT$, $TE$, $EE$, $BB$ and CMB lensing $\kappa\kappa$ power spectra are calculated with the `HDFisherFromPk` class in `hd_pk/cmb_from_pk.py`. 

First, we will describe each of the options available in the code (many of which are not required), which are set when initializing the class. Then we will give some examples of how to use it to calculate the numerical derivatives.

---

## NOTE: Calculating the derivatives in parallel using MPI

While MPI is not required, we _strongly_ recommend that you use MPI to calculate the numerical derivatives in parallel (especially if you are varying a large number of parameters). If you use MPI, you will need to run a Python script, rather than calculating the derivatives within a Jupyter notebook. We provide a python script, `run_examples.py`. See the instructions for how to run it under each example.

__If you would prefer to calculate the derivatives inside this notebook _without_ MPI__: set `use_mpi = False` below.

In [2]:
use_mpi = True

---

## Initialization of the `HDFisherFromPk` class

### Required arguments 

There is only a single _required_ argument that must be passed to the `HDFisherFromPk` class:
1. `output_dir` (`str`): the absolute path to the directory where you would like to save the output files.
    - You should use a new directory for each new set of derivatives you calculate.
    - Within this directory, two new directories will be created: one holding the theory spectra calculated when each parameter is varied up or down, and the other holding the derivatives of the spectra for each parameter.

If no other arguments are passed, the derivatives will be calculated for the $\Lambda$CDM + $N_\mathrm{eff}$ + $\sum m_\nu$ parameters in a CDM-only model, using the step sizes from [MacInnis & Sehgal (2024)](https://arxiv.org/abs/2405.12220).

### Optional arguments

The default behavior of the first three listed below assumes that you are also using the default fiducial parameter values and step sizes. See below for information about changing the parameter values and step sizes.

1. `baryonic_feedback` (`bool`, default is `False`): whether to include baryonic feedback effects.
    - By default, if `baryonic_feedback = True`, the baryonic feedback parameter of [HMcode](https://arxiv.org/abs/2009.01858), `'HMCode_logT_AGN'`, is varied with a fiducial value of `'HMCode_logT_AGN' = 7.8`.
2. `wdm` (`bool`, default is `False`): whether to use a warm dark matter (WDM) model, as opposed to cold dark matter (CDM), using our pre-computed transfer function.
    - By default, if `wdm = True`, the WDM mass `'m_wdm'` will be varied, with a fiducial value of `'m_wdm' = 1` (in keV).
3. `ksz` (`bool`, default is `False`): whether to include the kSZ effect in the temperature power spectrum.
    - By default, if `ksz = True`, the amplitude `'A_ksz'` and slope `'n_ksz'` of the kSZ power spectrum will be varied, with fiducial values of `'A_ksz' = 1` and `'n_ksz' = 0`.
4. `ksz_xfer_redshift` (`float` or `None`, default is `0.5`): if you set `ksz = True`, _and_ set _either_ `wdm = True` _or_ you are using your own transfer function (see below), the transfer function is applied to the kSZ spectrum at this fixed redshift.
    - If `ksz_xfer_redshift = None`, no transfer function is applied is the kSZ spectrum.
5. `cmb_types` (`list` of `str`, default is `['lensed', 'delensed']`): which kinds of CMB spectra to use; options include `'lensed'` and '`delensed'`.
6. `hd_data_version` (`str`, default is `'latest'`): which version of the CMB-HD data to use.
    - By default, the latest version of the data is used. The version used in MacInnis & Sehgal (2024) is version `'v1.1'`. See the [CMB-HD forecasting data](https://github.com/CMB-HD/hdMockData) repository for more information.
    - For the numerical derivatives, the data version determines the multipole ranges of the spectra and the lensing noise, which is used to calculate delensed CMB spectra.
  

#### Changing the set of parameters varied, their fiducial values, or their step sizes

For each parameter that is varied, we need to know its fiducial value, and its step size used to vary it up and down relative to this fiducial value. We also need to know the fiducial values of any fixed parameters that are needed to calculate the theory spectra, i.e., the parameters that we pass to CAMB (including accuracy and other CAMB parameters).


__Using your own fiducial parameter values__:

There are two ways to pass your own fiducial parameters values when initializing the `HDFisherFromPk` class: 
1. Using the `fiducial_params_dict` argument (`dict` or `None`, default is `None`): a dictionary with the parameter names as keys (`str`) and the values as the fiducial parameter values.
    - If `fiducial_params_dict = None`, the `fiducial_params_file` is used.
2. Using the `fiducial_params_file` argument (`str` or `None`, default is `None`): the absolute path to a YAML file holding the parameter names and values.
    - You can look at the files `hd_pk/fisher_param_files/fiducial_params.yaml` and `hd_pk/fisher_param_files/fiducial_params_feedback.yaml` for examples without and with baryonic feedback, respectively.
    - If `fiducial_params_file = None`, the default parameters described above are used.


In either case, you must include:
- _All_ parameters that should be passed to CAMB (cosmological parameters, accuracy parameters, etc.)
- If `baryonic_feedback = True`, you must provide a fiducial value for `'HMCode_logT_AGN'`, _and_ include CAMB's  `'halofit_version'` parameter with the value set to `'mead2020_feedback'`.
- If `ksz = True`, you must include fiducial values for `'A_ksz'` and `'n_ksz'`.
- If `wdm = True`, you must include a fiducial value for `'m_wdm'`.


__Using your own parameter step sizes__:

You only need to provide a step size for each parameter that you want to vary for the Fisher matrix; the others will remain fixed to their fiducial values.
- By default, we will calculate the numerical derviatives for each parameter that has a step size.
- Each parameter that has a step size must also have a fiducial value provided.


There are two ways to pass your own step sizes values when initializing the `HDFisherFromPk` class: 
1. Using the `fisher_steps_dict` argument (`dict` or `None`, default is `None`): a dictionary with the varied parameter names as keys (`str`) and their step sizes as the values.
    - If `fisher_steps_dict = None`, the `fisher_steps_file` is used.
2. Using the `fisher_steps_file` argument (`str` or `None`, default is `None`): the absolute path to a YAML file with two entries for each varied parameter name: a `step_type` (`str`), and a a `step_size` (`float`). 
    - If `fisher_steps_file = None`, we use the default step sizes from MacInnis & Sehgal (2024).


If you use a YAML file, you can specify each step size in one of two ways. An _absolute_ step size, which is a fixed value, or a _relative_ step size, taken as a fraction of the fiducial value. For example, if your fiducial value of `'ns'` is 0.96, and you want to vary it by 1% of this fiducial value, the relevant lines in your YAML file would be:
```
ns:
    step_type: relative
    step_size: 0.01
```
This is equivalent to specifying an absolute step size of $\pm$ 0.0096:
```
ns:
    step_type: absolute
    step_size: 0.0096
```
You can look at the files `hd_pk/fisher_param_files/fiducial_step_sizes.yaml` and `hd_pk/fisher_param_files/fiducial_step_sizes_feedback.yaml` for examples without and with baryonic feedback, respectively.


__Changing the list of varied parameters__:

You can pass a list of varied parmeter names to the `fisher_params` argument. This may be useful if you don't want to modify your dictionary or YAML holding the step sizes, but you only want to calculate derivatives for a subset of those parameters. By default, all parameters with a step size will be varied.


#### Using your own transfer function

To get a set of theory power spectra, we calculate the CMB lensing spectrum $C_L^{\kappa\kappa}$ from an integral over the non-linear matter power spectrum $P(k,~z)$ (obtained from CAMB), and pass this to CAMB to get the lensed CMB power spectra from the unlensed spectra. For the WDM models, we apply a transfer function $T^2_\mathrm{WDM}(k,z) \equiv P_\mathrm{WDM}(k,z) / P_\mathrm{CDM}(k,~z)$ to the CDM-only $P_\mathrm{CDM}(k,z)$ from CAMB.

We provide an option to use your own transfer function, $T^2(k,z) \equiv P_\mathrm{non-CDM}(k,z) / P_\mathrm{CDM}(k,~z)$. We also allow for this transfer function to depend on _any_ parameter that you want to vary. This means that you can vary any custom parameters of your model via the transfer function; but note that the calculations done by CAMB are independent of these custom parameters.

__Please see example 4 for more details__.

---

## Numerical derivatives examples

These examples show how to initialize the `HDFisherFromPk` class for different cases. In each case, the derivatives are calculated and saved using the `fisher_derivs` method of the `HDFisherFromPk` class.

### Example 1: CDM-only with the default fiducial parameter values and step sizes

In this case, you only need to provide an `output_dir`. If the `output_dir` does not already exist, it will be created. Within it, two new directories will be created: `theory`, holding the theory spectra used in the calculation, and `derivs`, holding the derivatives themselves. 

Below, we have set a default output directory, named `example01`, that will be placed inside the same directory as this notebook. You can change this to be the absolute path to whichever directory you'd like to use.

In [3]:
output_dir_example1 = os.path.join(os.getcwd(), 'example01')

Now we initialize the `HDFisherFromPk` class with this `output_dir`:

In [4]:
fisherlib_example1 = cmb_from_pk.HDFisherFromPk(output_dir_example1)

If you aren't using MPI to calculate the derivatives, then they will be calculated in the next cell. __Note__ that this could take over an hour, depending on your machine.

In [5]:
if not use_mpi:
    print('calculating the derivatives')
    fisherlib_example1.fisher_derivs()

If you are using MPI, you can run the example script provided as:

```
mpirun -np <N> python run_examples.py <output_dir> 1
```

where you should replace `<N>` with the number of parallel MPI processes to use (e.g., `4` or `12` or `24` or ...; this depends on your machine), and replacing `<output_dir>` with the absolute path of the `output_dir_example1` you defined above.

__Note__: if you are working on a cluster, see their documentation on using MPI; the command may differ from what's written above.

Once the derivatives have been calculated, you can move on to "step 2" below.

### Example 2: including the kSZ effect in a CDM-only model

#### Example 2a: using the fiducial kSZ amplitude and slope

The simplest way of including the kSZ effect is to pass `ksz = True`. If everything else is left to the default settings, then the two kSZ parameters `'A_ksz'` and `'n_ksz'` will automatically be varied.

We have set an `output_dir` below, but you can change this to be a different directory:

In [6]:
output_dir_example2a = os.path.join(os.getcwd(), 'example02a')
fisherlib_example2a = cmb_from_pk.HDFisherFromPk(output_dir_example2a, ksz=True)

In [7]:
if not use_mpi:
    print('calculating the derivatives')
    fisherlib_example2a.fisher_derivs()

If you are using MPI, you can run the example script provided as:

```
mpirun -np <N> python run_examples.py <output_dir> 2a
```

where you should replace `<N>` with the number of parallel MPI processes to use, and replacing `<output_dir>` with the absolute path of the `output_dir_example2a` you defined above.

#### Example 2b: changing the fiducial kSZ amplitude and slope

We model the kSZ power spectrum in CDM models as $C_\ell^\mathrm{kSZ} = A_\mathrm{kSZ} \left(\ell / 3000\right)^{n_\mathrm{kSZ}} C_\ell^{\mathrm{kSZ},0}$, where $C_\ell^{\mathrm{kSZ},0}$ is a template power spectrum. This template is fixed, but note that you can effectively use any template by changing the fiducial values of `'A_ksz'` and `'n_ksz'`. 

The easiest way to change _only_ the fiducial values of a few parameters, but otherwise use the default values, is to use the `default_fisher_params_step_sizes` function in `cmb_from_pk.py`. This takes in a single argument, `baryonic_feedback`, which can be `True` or `False` (default is `False`). It returns two dictionaries: the first holds the fiducial parameter names and values, and the second holds the varied parameter names and their step sizes. You can update either (or both) and pass them to the `HDFisherFromPk` class.

Below, we show how to change the fiducial value of `'n_ksz'` to be `-0.1`. __Note__ that, because we're overriding the default dictionaries (which include the WDM mass, `'m_wdm'`), we need to remove any unwanted parameters. 

In [8]:
default_fid_params, default_step_sizes = cmb_from_pk.default_fisher_params_step_sizes(baryonic_feedback=False)
print('All parameters that have a default fiducial value (no baryonic feedback): \n\t', list(default_fid_params.keys()))
print('All parameters that have a default step size (no baryonic feedback): \n\t', list(default_step_sizes.keys()))

All parameters that have a default fiducial value (no baryonic feedback): 
	 ['ombh2', 'omch2', 'theta', 'tau', 'logA', 'As', 'ns', 'nnu', 'mnu', 'num_massive_neutrinos', 'halofit_version', 'lmax', 'NonLinear', 'AccuracyBoost', 'lAccuracyBoost', 'lSampleBoost', 'DoLateRadTruncation', 'lens_margin', 'lens_potential_accuracy', 'A_ksz', 'n_ksz', 'm_wdm']
All parameters that have a default step size (no baryonic feedback): 
	 ['ombh2', 'omch2', 'theta', 'tau', 'logA', 'ns', 'nnu', 'mnu', 'A_ksz', 'n_ksz', 'm_wdm']


In [9]:
# make a copy of the defaults
example2b_fid_params = default_fid_params.copy()
example2b_step_sizes = default_step_sizes.copy()

# remove the WDM mass (not using it here):
example2b_fid_params.pop('m_wdm', None)
example2b_step_sizes.pop('m_wdm', None)

# update the fiducial value of n_ksz:
example2b_fid_params['n_ksz'] = -0.1

Now we initialize the `HDFisherFromPk` class with our new fiducial parameters and step sizes dictionaries, and run the derivatives:

In [10]:
output_dir_example2b = os.path.join(os.getcwd(), 'example02b')
fisherlib_example2b = cmb_from_pk.HDFisherFromPk(output_dir_example2b, ksz=True,
                                                fiducial_params_dict=example2b_fid_params,
                                                fisher_steps_dict=example2b_step_sizes)

In [11]:
if not use_mpi:
    print('calculating the derivatives')
    fisherlib_example2b.fisher_derivs()

If you are using MPI, you can run the example script provided as:

```
mpirun -np <N> python run_examples.py <output_dir> 2b
```

where you should replace `<N>` with the number of parallel MPI processes to use, and replacing `<output_dir>` with the absolute path of the `output_dir_example2b` you defined above.

### Example 3: non-CDM models, including the kSZ effect

#### Example 3a: CDM + baryonic feedback

To include baryonic feedback effects _and_ use the default parameter values and step sizes, you can just pass `baryonic_feedback=True` to the `HDFisherFromPk` class. 

__Note__ that if you use your own fiducial values and step sizes, you'll need to include the a fiducial value for the baryonic feedback parameter `'HMCode_logT_AGN'`, _and_ include CAMB's  `'halofit_version'` parameter with the value set to `'mead2020_feedback'`. We will just use the defaults in this example.

In [12]:
output_dir_example3a = os.path.join(os.getcwd(), 'example03a')
fisherlib_example3a = cmb_from_pk.HDFisherFromPk(output_dir_example3a, ksz=True, baryonic_feedback=True)

In [13]:
if not use_mpi:
    print('calculating the derivatives')
    fisherlib_example3a.fisher_derivs()

If you are using MPI, you can run the example script provided as:

```
mpirun -np <N> python run_examples.py <output_dir> 3a
```

where you should replace `<N>` with the number of parallel MPI processes to use, and replacing `<output_dir>` with the absolute path of the `output_dir_example3a` you defined above.

#### Example 3b: 1 keV warm dark matter (WDM)

To use a WDM-only model, you can pass `wdm=True` when initializing the `HDFisherFromPk` class. By default, the fiducial WDM mass is 1 keV. We will change this in example 3d below.

When also including the kSZ, we model the suppression in the kSZ power due to WDM by applying the WDM transfer function at a fixed redshift, using $z = 0.5$ by default (which we use below). You can change this with the `ksz_xfer_redshift` argument.

In [14]:
output_dir_example3b = os.path.join(os.getcwd(), 'example03b')
fisherlib_example3b = cmb_from_pk.HDFisherFromPk(output_dir_example3b, ksz=True, wdm=True, ksz_xfer_redshift=0.5)

In [15]:
if not use_mpi:
    print('calculating the derivatives')
    fisherlib_example3b.fisher_derivs()

If you are using MPI, you can run the example script provided as:

```
mpirun -np <N> python run_examples.py <output_dir> 3b
```

where you should replace `<N>` with the number of parallel MPI processes to use, and replacing `<output_dir>` with the absolute path of the `output_dir_example3b` you defined above.

#### Example 3c: 1 keV WDM + baryonic feedback

To include both WDM and baryonic feedback effects, you should set both `baryonic_feedback=True` and `wdm=True`. Note that we do not model the suppression in the kSZ due to baryonic feedback.

In [16]:
output_dir_example3c = os.path.join(os.getcwd(), 'example03c')
fisherlib_example3c = cmb_from_pk.HDFisherFromPk(output_dir_example3c, ksz=True, baryonic_feedback=True, wdm=True, ksz_xfer_redshift=0.5)

In [17]:
if not use_mpi:
    print('calculating the derivatives')
    fisherlib_example3c.fisher_derivs()

If you are using MPI, you can run the example script provided as:

```
mpirun -np <N> python run_examples.py <output_dir> 3c
```

where you should replace `<N>` with the number of parallel MPI processes to use, and replacing `<output_dir>` with the absolute path of the `output_dir_example3c` you defined above.

#### Example 3d: changing the WDM mass

We can change the WDM mass in the same way as we did for the slope of the kSZ in example 2b. Below, we will change it to 3 keV.

__Note__: We provide pre-computed WDM transfer functions for integer masses ranging from 1 to 12 keV. For each mass, we also provide two transfer functions, for each WDM mass varied up or down by 10% of its value. 
- This means that when you change the fiducial WDM mass, you should also change its step size - otherwise, you'll encounter an error message when the code tries to find the correct transfer function.
- Alternatively, if you're _only_ changing the WDM mass, you can still use the default step sizes (i.e., don't pass a dictionary or YAML file name); the default is a _relative_ step size of 10% for `'m_wdm'`.

In [18]:
default_fid_params_baryons, default_step_sizes_baryons = cmb_from_pk.default_fisher_params_step_sizes(baryonic_feedback=True)
print('All parameters that have a default fiducial value: \n\t', list(default_fid_params_baryons.keys()))
print('All parameters that have a default step size: \n\t', list(default_step_sizes_baryons.keys()))
print(f"\nDefault WDM mass +/- step size: ({default_fid_params_baryons['m_wdm']} +/- {default_step_sizes_baryons['m_wdm']}) keV")

All parameters that have a default fiducial value: 
	 ['ombh2', 'omch2', 'theta', 'tau', 'logA', 'As', 'ns', 'nnu', 'mnu', 'num_massive_neutrinos', 'halofit_version', 'HMCode_logT_AGN', 'lmax', 'NonLinear', 'AccuracyBoost', 'lAccuracyBoost', 'lSampleBoost', 'DoLateRadTruncation', 'lens_margin', 'lens_potential_accuracy', 'A_ksz', 'n_ksz', 'm_wdm']
All parameters that have a default step size: 
	 ['ombh2', 'omch2', 'theta', 'tau', 'logA', 'ns', 'nnu', 'mnu', 'HMCode_logT_AGN', 'A_ksz', 'n_ksz', 'm_wdm']

Default WDM mass +/- step size: (1 +/- 0.1) keV


In [19]:
# make a copy of the defaults
example3d_fid_params = default_fid_params_baryons.copy()
example3d_step_sizes = default_step_sizes_baryons.copy()

# update the fiducial WDM mass:
example3d_fid_params['m_wdm'] = 3 # keV
# update the step size to be 10% of the fiducial value:
example3d_step_sizes['m_wdm'] = 0.1 * example3d_fid_params['m_wdm']

Now, pass the new dictionaries when initializing the `HDFisherFromPk` class:

In [20]:
output_dir_example3d = os.path.join(os.getcwd(), 'example03d')
fisherlib_example3d = cmb_from_pk.HDFisherFromPk(output_dir_example3d, ksz=True,
                                                 baryonic_feedback=True, wdm=True, ksz_xfer_redshift=0.5,
                                                 fiducial_params_dict=example3d_fid_params,
                                                 fisher_steps_dict=example3d_step_sizes) 

In [21]:
if not use_mpi:
    print('calculating the derivatives')
    fisherlib_example3d.fisher_derivs()

If you are using MPI, you can run the example script provided as:

```
mpirun -np <N> python run_examples.py <output_dir> 3d
```

where you should replace `<N>` with the number of parallel MPI processes to use, and replacing `<output_dir>` with the absolute path of the `output_dir_example3d` you defined above.

### Example 4: using your own transfer function

As mentioned above, to calculate the CMB and CMB lensing power spectra in a non-CDM model, we apply a transfer function $T^2(k,z) \equiv P_\mathrm{non-CDM}(k,z) / P_\mathrm{CDM}(k,~z)$ to the $P_\mathrm{CDM}(k,~z)$ from CAMB, and calculate $C_L^{\kappa\kappa}$ by integrating $P(k,z)$ over the redshift range $z \in [0,1100]$. We use the approximation $k \approx (L + 1/2) / \chi(z)$ to relate the wavenumber $k$ to a redshift $z$ at a given lensing multipole $L$, where $\chi$ is the comoving distance.

#### How to store your transfer function file(s)

To obtain a transfer function that can be called at any $k$ and $z$ (within a given range of values), we interpolate it from a file that contains the transfer function evaluated at a set of redshifts and wavenumbers. In particular:
- The first column of the file should hold the wavenumbers $k$, in units of Mpc$^{-1}$, at which the transfer function is evaluated.
- The remaining columns should hold the transfer function evaluated at the wavenumbers in the first column and at a different, fixed redshift for each.
- The header of the file should contain the redshifts in the columns after the first, with each column heading separated by a comma. An example header would look like: `k [1/Mpc], z = 0, z = 1, z = 2`

__Note__: we take the redshift of the last column to be the $z_\mathrm{max}$ for the transfer function. We assume the transfer function is _independent of redshift_ for $z > z_\mathrm{max}$, and use the last column of your file as the redshift-independent transfer function. In other words, we use $T^2(k,z)$ for $z \leq z_\mathrm{max}$, and use $T^2(k) \equiv T^2(k,z_\mathrm{max})$ for $z > z_\mathrm{max}$.

#### How to initialize `HDFisherFromPk` with your transfer function

To use your own transfer function, you must _at least_ provide one file, in the format described above, holding the transfer function data calculated in your fiducial cosmology. When initializing the `HDFisherFromPk` class, you will pass the absolute path to the `xfer_dir` where this file is located. You must name the file `xfer_fid.txt`. If this is all that you provide, the _same_ transfer function will be applied for each theory calculation that is used to calculate the numerical derivatives.

If your transfer function depends on some set of parameters, you should also provide two files, in the same format as above, for each parameter: one calculated when that parameter is varied up by your step size, and one when the parameter is varied down. You must then pass a list of these parameter names to `xfer_params` when initializing the `HDFisherFromPk` class.

For example:
- Suppose you have a transfer function $T^2(k,z,\alpha)$ that depends on some model parameter named $\alpha$, and you want to vary this parameter away from a fiducial value of $\alpha_0$ with a step size of $\Delta\alpha$.
- There should be three files in the `xfer_dir`:
    1. `xfer_fid.txt`, holding the transfer function $T^2(k,z,\alpha_0)$ evaluated at the fiducial cosmology.
    2. `xfer_alpha_up.txt`, holding the transfer function $T^2(k,z,\alpha_0 + \Delta \alpha)$ when $\alpha$ is varied up.
    3. `xfer_alpha_down.txt`, holding the transfer function $T^2(k,z,\alpha_0 - \Delta \alpha)$ when $\alpha$ is varied down.
- You should pass the `xfer_dir`, along with `xfer_params = ['alpha']`, when initializing the `HDFisherFromPk` class.
- You should _also_ include `'alpha'` in your fiducial parameter dictionary or YAML file, with a fiducial value of $\alpha_0$, and its step size $\Delta \alpha$ in the step sizes dictionary or YAML file.
    - __Note__ that you are responsible for using consistent fiducial values and step sizes;  there is no way for the code to know if you used a different fiducial value or step size when calculating the transfer function vs. the numerical derivatives.
 
The example above can be extended to multiple parameters. 
- You can also include parameters that are passed to CAMB, so please be careful naming your parameters.
- __Note__ that, in addition to the CAMB parameter names, the code also uses the names `'theta'` for $100 \theta_\mathrm{MC}$ and `'logA'` for $\ln(10^{10} A_\mathrm{s})$. Please do not use these names for any other purpose.

#### Specific example: 1 keV WDM

As a specific example, we will use the same pre-computed WDM transfer function that is used when `wdm = True`. However, we'll show how to save it below, and then pass it to the `HDFisherFromPk` class as if it were any general transfer function. This transfer function depends on a single parameter, the WDM mass (in keV). We'll give this parameter a new name, `'wdm_mass'` instead of `'m_wdm'`.

First, we'll choose a directory where we will save the transfer functions. Below, this is set by default to be a directory `example_transfer` inside the directory where this notebook is, but you can feel free to change it.

In [22]:
xfer_dir = os.path.join(os.getcwd(), 'example_transfer')
# create the directory, if it doesn't already exist:
if not os.path.exists(xfer_dir):
    os.makedirs(xfer_dir)

Now we need to add some files to that directory. We'll start with the fiducial transfer function. We first need to define the range of wavenumbers and redshifts to use when evaluating the transfer function. We will use the non-linear WDM transfer function for $z \leq 14$, and the Viel et. al. linear transfer function above that. 

The function `wdm_transfer` in `cmb_from_pk.py` will return our WDM transfer function, including the linear function above $z = 14$.

In [23]:
xfer_ks = np.logspace(-3, 3, 1000)
xfer_zs = list(range(16)) # z = 15 will hold linear function

# choose the WDM mass:
wdm_mass = 1
# we also need to define Omega_WDM and h, for the linear transfer function:
h = 0.6736
Omega_wdm = 0.12 / h**2
# store arrays of the transfer function, evaluated at each redshift, in a list:
wdm_transfer_functions = []
for z in xfer_zs:
    wdm_transfer = cmb_from_pk.wdm_transfer(xfer_ks, z, wdm_mass, Omega_wdm, h)
    wdm_transfer_functions.append(wdm_transfer.copy())

# save the fiducial transfer function, with a header holding information about the redshifts:
header_col_names = ['k [Mpc^-1]'] + [f'z = {z}' for z in xfer_zs]
header = ', '.join(header_col_names)
save_cols = [xfer_ks] + [wdm_transfer_functions[z] for z in xfer_zs]
np.savetxt(os.path.join(xfer_dir, 'xfer_fid.txt'), np.column_stack(save_cols), header=header)

Now we vary the mass up and down by 10% of its fiducial value, and save those transfer functions also:

In [24]:
wdm_step_size = 0.1 * wdm_mass
wdm_mass_name = 'wdm_mass'

# repeat the same calculation, now varying the WDM mass parameter up and down by its step size:
for step_dir, mass in zip(['up', 'down'], [wdm_mass + wdm_step_size, wdm_mass - wdm_step_size]):
    # store arrays of the transfer function, evaluated at each redshift, in a list:
    wdm_transfer_functions = []
    for z in xfer_zs:
        wdm_transfer = cmb_from_pk.wdm_transfer(xfer_ks, z, mass, Omega_wdm, h)
        wdm_transfer_functions.append(wdm_transfer.copy())
    
    # save the fiducial transfer function, with a header holding information about the redshifts:
    header_col_names = ['k [Mpc^-1]'] + [f'z = {z}' for z in xfer_zs]
    header = ', '.join(header_col_names)
    save_cols = [xfer_ks] + [wdm_transfer_functions[z] for z in xfer_zs]
    np.savetxt(os.path.join(xfer_dir, f'xfer_{wdm_mass_name}_{step_dir}.txt'), np.column_stack(save_cols), header=header)

Now we pass the `xfer_dir` to the `HDFisherFromPk` class, along with the list of parameters the transfer function depends on; in this case, `xfer_params = ['wdm_mass']`. We also need to add this parameter to our dictionaries of fiducial parameters and step sizes, making sure we use the same values that were used to calculate the transfer function.

We will also include the kSZ effect, and apply the transfer function to it at $z = 0.5$ (which is the default).

In [25]:
output_dir_example4 = os.path.join(os.getcwd(), 'example04')
xfer_params = ['wdm_mass']

# copy the default parameter and step size dictionaries, and replace `'m_wdm'` with our `'wdm_mass'` parameter:
default_fid_params, default_step_sizes = cmb_from_pk.default_fisher_params_step_sizes(baryonic_feedback=True)
example4_fid_params = default_fid_params.copy()
example4_step_sizes = default_step_sizes.copy()
# remove the WDM mass parameter used with the precomputed transfer functions:
example4_fid_params.pop('m_wdm', None)
example4_step_sizes.pop('m_wdm', None)
# add our "new" parameter:
example4_fid_params['wdm_mass'] = wdm_mass
example4_step_sizes['wdm_mass'] = wdm_step_size

fisherlib_example4 = cmb_from_pk.HDFisherFromPk(output_dir_example4, 
                                                ksz=True, ksz_xfer_redshift=0.5,
                                                fiducial_params_dict=example4_fid_params,
                                                fisher_steps_dict=example4_step_sizes,
                                                xfer_dir=xfer_dir, xfer_params=xfer_params) 

In [26]:
if not use_mpi:
    print('calculating the derivatives')
    fisherlib_example4.fisher_derivs()

If you are using MPI, you can run the example script provided as:

```
mpirun -np <N> python run_examples.py <output_dir> 4 --xfer_dir=<xfer_dir>
```

where you should replace `<N>` with the number of parallel MPI processes to use, replacing `<output_dir>` with the absolute path of the `output_dir_example4` you defined above, and replacing `<xfer_dir>` with the absolute path to the `xfer_dir` you defined above.

---

---

# Step 2: calculating Fisher matrices

This is the easy part! 

If necessary, re-initialize the `HDFisherFromPk` in the same way you used when calculating the numerical derivatives. 

The `calc_fisher_matrix` method of the `HDFisherFromPk` class will automatically load those derivatives and the CMB-HD covariance matrix, and use them to calculate your fisher matrix. Alternatively, you can get the parameter error bars directly using the `get_fisher_errors` method. Both take in the same (optional) arguments:
1. `priors` (`dict` or `None`, default is `None`): a dictionary with parameter names as keys are their _Gaussian_ prior as the value.
    - __By default, no prior is applied. We strongly recommend using a prior on $\tau$ of $\sigma(\tau) = 0.007$__ from _Planck_. When including baryonic feedback, we also recommend applying a 0.06% prior on $\log_{10}(T_\mathrm{AGN}/\mathrm{K})$. We use both of these priors below.
2. `params` (`list` of `str` or `None`; default is `None`): a list of parameter names to include in the Fisher matrix. By default, all parameters that were varied are included.
3. `desi_bao` (`bool`, default is `False`): whether to combine the CMB-HD Fisher matrix with a mock DESI BAO Fisher matrix.
4. `spectra` (`list` of `str` or `None`; default is `None`): the kinds of spectra to include in the Fisher matrix calculation. The options are `'tt'`, `'te'`, `'ee'`, `'bb'`, `'kk'` for the lensed/delensed CMB $TT$, $TE$, $EE$, $BB$ and CMB lensing $\kappa\kappa$ spectra, respectively. By default, all are used.

The `calc_fisher_matrix` method returns a tuple of dictionaries: one holding the fisher matrices, and the second holding the list of parameters in the Fisher matrix, with the order corresponding to their order along the rows/columns of the matrix. Each dictionary has a key for each `cmb_type` (i.e. `'lensed'` and `'delensed'`) that was requested when initializing the `HDFisherFromPk` class. 

The `get_fisher_errors` method returns a single, nested dictionary. The first set of keys are the `cmb_types` (`'lensed'` and `'delensed'`). The dictionary for each `cmb_type` holds the parameter names as keys and their 1$\sigma$ uncertainties as the values.

---

Below, we will go through each example listed above. When applicable, we will compare the results with the precomputed results from [MacInnis & Sehgal (2024)](https://arxiv.org/abs/2405.12220). In each case, we will re-initialize the `HDFisherFromPk` class; if you changed the `output_dir` when you calculated the derivatives, you'll also need to change it below (for each example you ran).
- __Note__: the results in MacInnis & Sehgal (2024) use `hd_data_version = 'v1.1'`, but the default version when initializing the `HDFisherFromPk` is set to be the latest version. If you have trouble matching the precomputed results we provide, make sure you're using the correct version of the data.
- Also note that the _precomputed_ Fisher matrices already have the $\tau$ prior applied, but they do not include the SZ prior on the baryonic feedback parameter.

First, we will define some constants that are the same in each case, and define some functions that will be useful when printing out the results.

In [27]:
priors = {'tau': 0.007}
sz_prior = {'HMCode_logT_AGN': 0.0006 * 7.8} # 0.06% prior 
priors_baryons = {**priors, **sz_prior}
desi_bao = True
cmb_type = 'delensed'

# number of digits to use when rounding the uncertainty of each parameter:
ndigits = {'ombh2': 7, 'omch2': 6, 'logA': 5, 'ns': 5, 'tau': 5, 'theta': 7,
           'nnu': 4, 'mnu': 4, 'HMCode_logT_AGN': 5, 'm_wdm': 4, 'A_ksz': 6,  'n_ksz': 6,
          'wdm_mass': 4}

def round_dict(d, ndigits=ndigits):
    """Given a dictionary `d` with numerical values, round each value 
    to the number of digits for that key, given by the `ndigits` dict."""
    rounded = {}
    for key, val in d.items():
        rounded[key] = round(val, ndigits[key])
    return rounded

def ratio(dict1, dict2, fill_value=None):
    """Returns a dictionary with the ratio of the values in `dict1` to those 
    in `dict2` for each key they share in common. If the keys don't match, you
    can pass a `fill_value` as a placeholder for any keys not in both dictionaries;
    otherwise leave it as `None` to exclude those keys from the returned dictionary."""
    rdict = {}
    keys1 = list(dict1.keys())
    keys2 = list(dict2.keys())
    all_keys = list(set(keys1 + keys2))
    for key in all_keys:
        if (key in dict1) and (key in dict2):
            rdict[key] = dict1[key] / dict2[key]
        elif fill_value is not None:
            rdict[key] = fill_value
    return rdict

def print_table(list_of_dicts, list_of_labels, title=None, list_of_keys=None):
    """
    Display a table of keys and values held in each dictionary in the `list_of_dicts`.

    list_of_dicts : a list of dictionaries; each is printed as a column, with the keys as rows.
    list_of_labels : a list holding a column label for each dictionary in `list_of_dicts`, in
                     the same order that they appear in `list_of_dicts`.
    title : a title that is printed out above the table.
    list_of_keys : a list of keys to use as the rows in the table, in the same order as they appear in the list. 
    """
    table = pd.DataFrame(list_of_dicts, index=list_of_labels, columns=list_of_keys)
    table = table.T
    title = ' ' if title is None else title
    table.style.set_caption(title).set_table_styles([{'selector': 'caption', 'props': [('color', 'k'), ('font-size', '26px')]}])
    print('\n', title) 
    display(table)

## Example 1

Re-initialize the `HDFisherFromPk` class, changing the `output_dir` if necessary:

In [28]:
output_dir_example1 = os.path.join(os.getcwd(), 'example01')
fisherlib_example1 = cmb_from_pk.HDFisherFromPk(output_dir_example1)

Get the parameter errors:

In [29]:
errors_example1 = fisherlib_example1.get_fisher_errors(priors=priors, desi_bao=desi_bao)

And we will load in the precomputed Fisher matrix for comparison:

In [30]:
precomputed_fisher_example1, fisher_params_example1 = cmb_from_pk.load_precomputed_fisher_matrix(cmb_type, with_desi=True)
precomputed_errors_example1 = fisher.get_fisher_errors(precomputed_fisher_example1, fisher_params_example1)

# compare them
print_table([round_dict(errors_example1[cmb_type]), round_dict(precomputed_errors_example1), 
             ratio(round_dict(errors_example1[cmb_type]), round_dict(precomputed_errors_example1))],
            ['(1) Your calculation', '(2) Precomputed', 'Ratio: (1) / (2)'])

,(1) Your calculation,(2) Precomputed,Ratio: (1) / (2)
logA,0.009790,0.009790,1.0
mnu,0.024500,0.024500,1.0
nnu,0.013700,0.013700,1.0
ns,0.001440,0.001440,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000407,0.000407,1.0
tau,0.005230,0.005230,1.0
theta,0.000059,0.000059,1.0


## Example 2a

Re-initialize the `HDFisherFromPk` class, changing the `output_dir` if necessary:

In [31]:
output_dir_example2a = os.path.join(os.getcwd(), 'example02a')
fisherlib_example2a = cmb_from_pk.HDFisherFromPk(output_dir_example2a, ksz=True)

Get the parameter errors:

In [32]:
errors_example2a = fisherlib_example2a.get_fisher_errors(priors=priors, desi_bao=desi_bao)

And we will load in the precomputed Fisher matrix for comparison:

In [33]:
precomputed_fisher_example2a, fisher_params_example2a = cmb_from_pk.load_precomputed_fisher_matrix(cmb_type, with_desi=True, with_ksz=True)
precomputed_errors_example2a = fisher.get_fisher_errors(precomputed_fisher_example2a, fisher_params_example2a)

# compare them
print_table([round_dict(errors_example2a[cmb_type]), round_dict(precomputed_errors_example2a), 
             ratio(round_dict(errors_example2a[cmb_type]), round_dict(precomputed_errors_example2a))],
            ['(1) Your calculation', '(2) Precomputed', 'Ratio: (1) / (2)'])

,(1) Your calculation,(2) Precomputed,Ratio: (1) / (2)
A_ksz,0.000762,0.000762,1.0
logA,0.010370,0.010370,1.0
mnu,0.026500,0.026500,1.0
n_ksz,0.000492,0.000492,1.0
nnu,0.015200,0.015200,1.0
ns,0.001560,0.001560,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000414,0.000414,1.0
tau,0.005490,0.005490,1.0
theta,0.000060,0.000060,1.0


## Example 2b

Re-initialize the `HDFisherFromPk` class, changing the `output_dir` if necessary:

In [34]:
output_dir_example2b = os.path.join(os.getcwd(), 'example02b')
# update the fiducial value of n_ksz:
default_fid_params, default_step_sizes = cmb_from_pk.default_fisher_params_step_sizes(baryonic_feedback=False)
example2b_fid_params = default_fid_params.copy()
example2b_step_sizes = default_step_sizes.copy()
example2b_fid_params.pop('m_wdm', None)
example2b_step_sizes.pop('m_wdm', None)
example2b_fid_params['n_ksz'] = -0.1
fisherlib_example2b = cmb_from_pk.HDFisherFromPk(output_dir_example2b, ksz=True,
                                                fiducial_params_dict=example2b_fid_params,
                                                fisher_steps_dict=example2b_step_sizes)

Get the parameter errors:

In [35]:
errors_example2b = fisherlib_example2b.get_fisher_errors(priors=priors, desi_bao=desi_bao)

We don't have a precomputed Fisher matrix in this case, so we will just print out the results:

In [36]:
print_table([round_dict(errors_example2b[cmb_type])], ['Your calculation'])

,Your calculation
A_ksz,0.000832
logA,0.010370
mnu,0.026500
n_ksz,0.000554
nnu,0.015200
ns,0.001570
ombh2,0.000025
omch2,0.000414
tau,0.005490
theta,0.000060


## Example 3a

Re-initialize the `HDFisherFromPk` class, changing the `output_dir` if necessary:

In [37]:
output_dir_example3a = os.path.join(os.getcwd(), 'example03a')
fisherlib_example3a = cmb_from_pk.HDFisherFromPk(output_dir_example3a, ksz=True, baryonic_feedback=True)

Get the parameter errors, applying the prior on the baryonic feedback parameter:

In [38]:
errors_example3a = fisherlib_example3a.get_fisher_errors(priors=priors_baryons, desi_bao=desi_bao)

And we will load in the precomputed Fisher matrix for comparison:

In [39]:
precomputed_fisher_example3a, fisher_params_example3a = cmb_from_pk.load_precomputed_fisher_matrix(cmb_type, with_desi=True, with_ksz=True, baryonic_feedback=True)
# apply the SZ prior
precomputed_fisher_example3a = fisher.add_priors(precomputed_fisher_example3a, fisher_params_example3a, sz_prior)
precomputed_errors_example3a = fisher.get_fisher_errors(precomputed_fisher_example3a, fisher_params_example3a)

# compare them
print_table([round_dict(errors_example3a[cmb_type]), round_dict(precomputed_errors_example3a), 
             ratio(round_dict(errors_example3a[cmb_type]), round_dict(precomputed_errors_example3a))],
            ['(1) Your calculation', '(2) Precomputed', 'Ratio: (1) / (2)'])

,(1) Your calculation,(2) Precomputed,Ratio: (1) / (2)
A_ksz,0.000750,0.000750,1.0
HMCode_logT_AGN,0.004650,0.004650,1.0
logA,0.010900,0.010900,1.0
mnu,0.027900,0.027900,1.0
n_ksz,0.000491,0.000491,1.0
nnu,0.015600,0.015600,1.0
ns,0.001600,0.001600,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000409,0.000409,1.0
tau,0.005770,0.005770,1.0


## Example 3b

Re-initialize the `HDFisherFromPk` class, changing the `output_dir` if necessary:

In [40]:
output_dir_example3b = os.path.join(os.getcwd(), 'example03b')
fisherlib_example3b = cmb_from_pk.HDFisherFromPk(output_dir_example3b, ksz=True, wdm=True, ksz_xfer_redshift=0.5)

Get the parameter errors:

In [41]:
errors_example3b = fisherlib_example3b.get_fisher_errors(priors=priors, desi_bao=desi_bao)

And we will load in the precomputed Fisher matrix for comparison:

In [42]:
precomputed_fisher_example3b, fisher_params_example3b = cmb_from_pk.load_precomputed_fisher_matrix(cmb_type, with_desi=True, with_ksz=True, m_wdm=1)
precomputed_errors_example3b = fisher.get_fisher_errors(precomputed_fisher_example3b, fisher_params_example3b)

# compare them
print_table([round_dict(errors_example3b[cmb_type]), round_dict(precomputed_errors_example3b), 
             ratio(round_dict(errors_example3b[cmb_type]), round_dict(precomputed_errors_example3b))],
            ['(1) Your calculation', '(2) Precomputed', 'Ratio: (1) / (2)'])

,(1) Your calculation,(2) Precomputed,Ratio: (1) / (2)
A_ksz,0.001219,0.001219,1.0
logA,0.010550,0.010550,1.0
m_wdm,0.033900,0.033900,1.0
mnu,0.026900,0.026900,1.0
n_ksz,0.001332,0.001332,1.0
nnu,0.016200,0.016200,1.0
ns,0.001690,0.001690,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000415,0.000415,1.0
tau,0.005590,0.005590,1.0


## Example 3c

Re-initialize the `HDFisherFromPk` class, changing the `output_dir` if necessary:

In [43]:
output_dir_example3c = os.path.join(os.getcwd(), 'example03c')
fisherlib_example3c = cmb_from_pk.HDFisherFromPk(output_dir_example3c, ksz=True, baryonic_feedback=True, wdm=True, ksz_xfer_redshift=0.5)

Get the parameter errors, applying the prior on the baryonic feedback parameter:

In [44]:
errors_example3c = fisherlib_example3c.get_fisher_errors(priors=priors_baryons, desi_bao=desi_bao)

And we will load in the precomputed Fisher matrix for comparison:

In [45]:
precomputed_fisher_example3c, fisher_params_example3c = cmb_from_pk.load_precomputed_fisher_matrix(cmb_type, with_desi=True, with_ksz=True, baryonic_feedback=True, m_wdm=1)
# apply the SZ prior
precomputed_fisher_example3c = fisher.add_priors(precomputed_fisher_example3c, fisher_params_example3c, sz_prior)
precomputed_errors_example3c = fisher.get_fisher_errors(precomputed_fisher_example3c, fisher_params_example3c)

# compare them
print_table([round_dict(errors_example3c[cmb_type]), round_dict(precomputed_errors_example3c), 
             ratio(round_dict(errors_example3c[cmb_type]), round_dict(precomputed_errors_example3c))],
            ['(1) Your calculation', '(2) Precomputed', 'Ratio: (1) / (2)'])

,(1) Your calculation,(2) Precomputed,Ratio: (1) / (2)
A_ksz,0.001214,0.001214,1.0
HMCode_logT_AGN,0.004650,0.004650,1.0
logA,0.010970,0.010970,1.0
m_wdm,0.033800,0.033800,1.0
mnu,0.028000,0.028000,1.0
n_ksz,0.001333,0.001333,1.0
nnu,0.016400,0.016400,1.0
ns,0.001710,0.001710,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000410,0.000410,1.0


## Example 3d

Re-initialize the `HDFisherFromPk` class, changing the `output_dir` if necessary:

In [46]:
output_dir_example3d = os.path.join(os.getcwd(), 'example03d')
# update the fiducial WDM mass:
default_fid_params_baryons, default_step_sizes_baryons = cmb_from_pk.default_fisher_params_step_sizes(baryonic_feedback=True)
example3d_fid_params = default_fid_params_baryons.copy()
example3d_step_sizes = default_step_sizes_baryons.copy()
example3d_fid_params['m_wdm'] = 3 # keV
example3d_step_sizes['m_wdm'] = 0.1 * example3d_fid_params['m_wdm']
fisherlib_example3d = cmb_from_pk.HDFisherFromPk(output_dir_example3d, ksz=True,
                                                 baryonic_feedback=True, wdm=True, ksz_xfer_redshift=0.5,
                                                 fiducial_params_dict=example3d_fid_params,
                                                 fisher_steps_dict=example3d_step_sizes) 

Get the parameter errors, applying the prior on the baryonic feedback parameter:

In [47]:
errors_example3d = fisherlib_example3d.get_fisher_errors(priors=priors_baryons, desi_bao=desi_bao)

And we will load in the precomputed Fisher matrix for comparison:

In [48]:
precomputed_fisher_example3d, fisher_params_example3d = cmb_from_pk.load_precomputed_fisher_matrix(cmb_type, with_desi=True, with_ksz=True, baryonic_feedback=True, m_wdm=3)
# apply the SZ prior
precomputed_fisher_example3d = fisher.add_priors(precomputed_fisher_example3d, fisher_params_example3d, sz_prior)
precomputed_errors_example3d = fisher.get_fisher_errors(precomputed_fisher_example3d, fisher_params_example3d)

# compare them
print_table([round_dict(errors_example3d[cmb_type]), round_dict(precomputed_errors_example3d), 
             ratio(round_dict(errors_example3d[cmb_type]), round_dict(precomputed_errors_example3d))],
            ['(1) Your calculation', '(2) Precomputed', 'Ratio: (1) / (2)'])

,(1) Your calculation,(2) Precomputed,Ratio: (1) / (2)
A_ksz,0.001125,0.001125,1.0
HMCode_logT_AGN,0.004650,0.004650,1.0
logA,0.010940,0.010940,1.0
m_wdm,0.306000,0.306000,1.0
mnu,0.027900,0.027900,1.0
n_ksz,0.001567,0.001567,1.0
nnu,0.015700,0.015700,1.0
ns,0.001620,0.001620,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000410,0.000410,1.0


## Example 4

Re-initialize the `HDFisherFromPk` class, changing the `output_dir` and `xfer_dir` if necessary:

In [49]:
output_dir_example4 = os.path.join(os.getcwd(), 'example04')
xfer_dir = os.path.join(os.getcwd(), 'example_transfer')
xfer_params = ['wdm_mass']

# copy the default parameter and step size dictionaries, and replace `'m_wdm'` with our `'wdm_mass'` parameter:
default_fid_params, default_step_sizes = cmb_from_pk.default_fisher_params_step_sizes(baryonic_feedback=True)
example4_fid_params = default_fid_params.copy()
example4_step_sizes = default_step_sizes.copy()
# remove the WDM mass parameter used with the precomputed transfer functions:
example4_fid_params.pop('m_wdm', None)
example4_step_sizes.pop('m_wdm', None)
# add our "new" parameter:
example4_fid_params['wdm_mass'] = wdm_mass
example4_step_sizes['wdm_mass'] = wdm_step_size

fisherlib_example4 = cmb_from_pk.HDFisherFromPk(output_dir_example4, 
                                                ksz=True, ksz_xfer_redshift=0.5,
                                                fiducial_params_dict=example4_fid_params,
                                                fisher_steps_dict=example4_step_sizes,
                                                xfer_dir=xfer_dir, xfer_params=xfer_params) 

Get the parameter errors, applying the prior on the baryonic feedback parameter:

In [50]:
errors_example4 = fisherlib_example4.get_fisher_errors(priors=priors_baryons, desi_bao=desi_bao)

First we will print out the error bars, to see that the new parameter name `'wdm_mass'` was used. Then we'll change it back to `'m_wdm'` in order to compare the new calculation with the pre-computed results.

In [51]:
print_table([round_dict(errors_example4[cmb_type])], ['Your calculation'])

,Your calculation
A_ksz,0.001214
HMCode_logT_AGN,0.004650
logA,0.010970
mnu,0.028000
n_ksz,0.001333
nnu,0.016400
ns,0.001710
ombh2,0.000025
omch2,0.000410
tau,0.005800


Now compare with the precomputed results:

In [52]:
errors_example4[cmb_type]['m_wdm'] = errors_example4[cmb_type]['wdm_mass']
errors_example4[cmb_type].pop('wdm_mass', None)

precomputed_fisher_example4, fisher_params_example4 = cmb_from_pk.load_precomputed_fisher_matrix(cmb_type, with_desi=True, with_ksz=True, baryonic_feedback=True, m_wdm=1)
# apply the SZ prior
precomputed_fisher_example4 = fisher.add_priors(precomputed_fisher_example4, fisher_params_example4, sz_prior)
precomputed_errors_example4 = fisher.get_fisher_errors(precomputed_fisher_example4, fisher_params_example4)

# compare them
print_table([round_dict(errors_example4[cmb_type]), round_dict(precomputed_errors_example4), 
             ratio(round_dict(errors_example4[cmb_type]), round_dict(precomputed_errors_example4))],
            ['(1) Your calculation', '(2) Precomputed', 'Ratio: (1) / (2)'])

,(1) Your calculation,(2) Precomputed,Ratio: (1) / (2)
A_ksz,0.001214,0.001214,1.0
HMCode_logT_AGN,0.004650,0.004650,1.0
logA,0.010970,0.010970,1.0
mnu,0.028000,0.028000,1.0
n_ksz,0.001333,0.001333,1.0
nnu,0.016400,0.016400,1.0
ns,0.001710,0.001710,1.0
ombh2,0.000025,0.000025,1.0
omch2,0.000410,0.000410,1.0
tau,0.005800,0.005800,1.0
